In [1]:
#Business Problem
# Support (X , Y) = Freq (X,Y)/N
# Confidence(X , Y) = Freq (X , Y) / Freq (X)
# Lift = Support ( X , Y) / (Support (X) * Support (Y) )

In [2]:
#pip install mlxtend

In [3]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules

In [4]:
df = pd.read_csv("retail_dataset.csv" , sep=",")
df.head()

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


In [6]:
df.shape

(315, 7)

In [7]:
items = (df["0"].unique())
items

array(['Bread', 'Cheese', 'Meat', 'Eggs', 'Wine', 'Bagel', 'Pencil',
       'Diaper', 'Milk'], dtype=object)

In [8]:
# one-hot encoding
encoded_vals = []
for index, row in df.iterrows():
    labels = {}
    uncommons = list(set(items) - set(row))
    commons = list(set(items).intersection(row))
    for uc in uncommons:
        labels[uc] = 0
    for com in commons:
        labels[com] = 1
    encoded_vals.append(labels)
encoded_vals[0]
ohe_df = pd.DataFrame(encoded_vals)        
# Sütunların 0 ve 1 olarak doldurulması için fonksiyon

In [11]:
ohe_df = pd.DataFrame(encoded_vals)
ohe_df.head()

,Milk,Bagel,Cheese,Pencil,Wine,Diaper,Bread,Meat,Eggs
0,0,0,1,1,1,1,1,1,1
1,1,0,1,1,1,1,1,1,0
2,1,0,1,0,1,0,0,1,1
3,1,0,1,0,1,0,0,1,1
4,0,0,0,1,1,0,0,1,0


In [12]:
freq_items = apriori(ohe_df, min_support = 0.1, use_colnames = True, verbose = 1)
#freq_items.value_counts("support" , ascending = False)
freq_items

# support : 2 tane ürünün ya da 1 tane ürünün satıl alınma olasılığı

Processing 5 combinations | Sampling itemset size 5


/Users/ozlemmunar/opt/anaconda3/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.501587,(Milk)
1,0.425397,(Bagel)
2,0.501587,(Cheese)
3,0.361905,(Pencil)
4,0.438095,(Wine)
...,...,...
95,0.120635,"(Diaper, Bread, Meat)"
96,0.101587,"(Milk, Cheese, Meat, Wine)"
97,0.104762,"(Milk, Cheese, Eggs, Wine)"
98,0.152381,"(Milk, Cheese, Meat, Eggs)"


In [14]:
association_rules(freq_items, metric="confidence" , min_threshold = 0.3)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Milk),(Bagel),0.501587,0.425397,0.225397,0.449367,1.056348,0.012023,1.043532
1,(Bagel),(Milk),0.425397,0.501587,0.225397,0.529851,1.056348,0.012023,1.060116
2,(Milk),(Cheese),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148
3,(Cheese),(Milk),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148
4,(Milk),(Pencil),0.501587,0.361905,0.171429,0.341772,0.944370,-0.010098,0.969414
...,...,...,...,...,...,...,...,...,...
308,"(Cheese, Eggs)","(Meat, Wine)",0.298413,0.250794,0.111111,0.372340,1.484649,0.036271,1.193651
309,"(Cheese, Wine)","(Meat, Eggs)",0.269841,0.266667,0.111111,0.411765,1.544118,0.039153,1.246667
310,"(Meat, Eggs)","(Cheese, Wine)",0.266667,0.269841,0.111111,0.416667,1.544118,0.039153,1.251701
311,"(Meat, Wine)","(Cheese, Eggs)",0.250794,0.298413,0.111111,0.443038,1.484649,0.036271,1.259668


In [18]:
df_ar = association_rules(freq_items, metric="confidence" , min_threshold = 0.3)

In [19]:
df_ar[(df_ar.support < 0.3) & (df_ar.confidence > 0.7)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
73,"(Milk, Bagel)",(Bread),0.225397,0.504762,0.171429,0.760563,1.506777,0.057657,2.068347
82,"(Milk, Wine)",(Cheese),0.219048,0.501587,0.161905,0.739130,1.473583,0.052033,1.910582
91,"(Milk, Meat)",(Cheese),0.244444,0.501587,0.203175,0.831169,1.657077,0.080564,2.952137
97,"(Milk, Eggs)",(Cheese),0.244444,0.501587,0.196825,0.805195,1.605293,0.074215,2.558519
116,"(Milk, Diaper)",(Bread),0.155556,0.504762,0.114286,0.734694,1.455526,0.035767,1.866667
125,"(Milk, Meat)",(Eggs),0.244444,0.438095,0.177778,0.727273,1.660079,0.070688,2.060317
126,"(Milk, Eggs)",(Meat),0.244444,0.476190,0.177778,0.727273,1.527273,0.061376,1.920635
214,"(Cheese, Eggs)",(Meat),0.298413,0.476190,0.215873,0.723404,1.519149,0.073772,1.893773
215,"(Meat, Eggs)",(Cheese),0.266667,0.501587,0.215873,0.809524,1.613924,0.082116,2.616667
230,"(Pencil, Eggs)",(Wine),0.165079,0.438095,0.120635,0.730769,1.668060,0.048314,2.087075


In [20]:
df_ar.to_csv("OutPut.csv")